# Data Generation by Simulation

In [1]:
#초기 세팅
#지수분포로 차의 분포 생성
import numpy as np
import math
timeBetweenCars = 0 #차 간 시간차이의 최솟값
carsArriveTime = np.zeros([8,15])
time_of_signal = 10 #각 신호의 지속 시간
timelimit = 40 #측정 시간. 각 신호의 지속 시간의 배수로 잡는다. 
signal_num = int(timelimit/time_of_signal) #신호의 갯수
for j in range(8):
    x = np.random.exponential(6,size = 15) #1/lambda = 6, 10개의 차량 , w1, w2-w1, ;.. w10 - w9
    y = []
    for i in range(15):
        y.append(sum(x[:i+1]+timeBetweenCars*i)) #시간차들을 더해 줘야 시간이 나옴
    carsArriveTime[j,:] = y #도착시간 행렬 8*10 행렬 (10대의 차량)
signal = [3, 5, 1, 6] #신호
cars_remaining = [[]]*8
time_crossing = 3# 각 차가 교차로를 지나가는 데 걸리는 시간 
def choose(signal_at_each_time): #각 신호에 따른 허용된 교통 흐름 
    if signal_at_each_time == 0:
        return [0,1]
    elif signal_at_each_time == 1:
        return [0,2]
    elif signal_at_each_time == 2:
        return [2,3]
    elif signal_at_each_time == 3:
        return [1,3]
    elif signal_at_each_time == 4:
        return [4,5]
    elif signal_at_each_time == 5:
        return [4,6]
    elif signal_at_each_time == 6:
        return [6,7]
    elif signal_at_each_time == 7:
        return [5,7]
def generate_signal(n): #주어진 n을 8진수로 변환해 신호 생성
    signal = []
    for l in str(oct(n))[2:]:
        signal.append(int(l))
    while(len(signal)<signal_num):
        signal.insert(0,0)
    return signal
def getMinSignal_square(carsArriveTime, signal_num,time_of_signal):
    mintime = 1e10
    for k in range(pow(8,signal_num)):
        signal = generate_signal(k)
        cars_remaining = [[]]*8
        Total_time_square = 0 #cost function: (각 차가 목적지까지 도달하는 시간 - tie_crossing)의 제곱의 합 
        for i in range(signal_num):
            count1 = 0
            current_movable_cars_flow = choose(signal[i]) #현재 신호에서 이동 가능한 차들 흐름 
            remain1,remain2 = cars_remaining[current_movable_cars_flow[0]],cars_remaining[current_movable_cars_flow[1]]
            for j in range(8):
                remaining_temp = [k for k in carsArriveTime[j,:] if (k<time_of_signal*(i+1) and k>time_of_signal*i)]
                cars_remaining[j] = cars_remaining[j] + remaining_temp #남은 차 추가, 이 때 빠져나가는 차들은 밑에서 처리.
            cars_remaining[current_movable_cars_flow[0]] = [];cars_remaining[current_movable_cars_flow[1]] = []#지금 신호에서 남은 자동차들 전부 빠져나간다
            remain1 = time_of_signal*i - np.asfarray(remain1)
            remain2 = time_of_signal*i - np.asfarray(remain2)
            Total_time_square = Total_time_square + float(np.dot(remain1,remain1)) + float(np.dot(remain2,remain2))
        for i in range(8):
            remain_final = cars_remaining[i] 
            remain_final = timelimit - np.asfarray(remain_final)
            Total_time_square = Total_time_square + np.dot(remain_final,remain_final)
        if(Total_time_square<mintime):
            minsignal = signal
            mintime = Total_time_square
    return minsignal + [mintime]
def getMinSignal_sum(carsArriveTime, signal_num,time_of_signal):
    mintime = 1e10
    for k in range(pow(8,signal_num)):
        signal = generate_signal(k)
        cars_remaining = [[]]*8
        Total_time_sum = 0 #cost function: (각 차가 목적지까지 도달하는 시간 - tie_crossing)의 제곱의 합 
        for i in range(signal_num):
            count1 = 0
            current_movable_cars_flow = choose(signal[i]) #현재 신호에서 이동 가능한 차들 흐름 
            remain1,remain2 = cars_remaining[current_movable_cars_flow[0]],cars_remaining[current_movable_cars_flow[1]]
            for j in range(8):
                remaining_temp = [k for k in carsArriveTime[j,:] if (k<time_of_signal*(i+1) and k>time_of_signal*i)]
                cars_remaining[j] = cars_remaining[j] + remaining_temp #남은 차 추가, 이 때 빠져나가는 차들은 밑에서 처리.
            cars_remaining[current_movable_cars_flow[0]] = [];cars_remaining[current_movable_cars_flow[1]] = []#지금 신호에서 남은 자동차들 전부 빠져나간다
            Total_time_sum = Total_time_sum + time_of_signal*i*len(remain1) - sum(remain1) + time_of_signal*i*len(remain2) - sum(remain2)
        for i in range(8):
            remain_final = cars_remaining[i] 
            Total_time_sum = Total_time_sum + timelimit*len(remain_final) - sum(remain_final)
        if(Total_time_sum<mintime):
            minsignal = signal
            mintime = Total_time_sum
    return minsignal + [mintime]
print(carsArriveTime)

[[  9.610543    22.3908824   23.0301819   30.23625045  33.48753307
   37.63479997  42.6609377   58.87912097  58.98435246  80.97218417
   88.55209725  91.96830634  99.19175227  99.88123635 100.60381014]
 [  3.04822462  15.59165943  26.30392347  46.25605023  51.79832457
   62.2034685   67.40938078  68.14801067  75.36400003  81.4659407
   81.66225597  85.70096718  87.57194869  88.3094915   95.87154021]
 [  0.29088441   5.72027526   8.85001219  14.96635827  24.76495262
   27.98197109  28.28353071  28.60760016  29.50255242  37.95811218
   39.57652763  40.65448783  40.66892766  48.71376045  50.42112465]
 [  6.36373608  12.5219892   25.44244949  30.02648272  31.07519896
   36.80319105  37.06408675  42.43877198  50.05437818  66.21400488
   77.91608795  79.19893661  99.47032887  99.7401808  100.75931351]
 [  4.3770577    6.41939313  11.81131502  25.20411637  32.59814293
   33.07556425  40.65863593  45.58684125  52.66852884  53.26583872
   56.04570461  57.49403108  60.42318413  60.84998009  71.2

# write data to csv file

In [5]:
import csv
f = open('output_raw.csv', 'a', encoding='utf-8', newline='')
wr = csv.writer(f)
carsArriveTime = np.zeros([8,55])
n_data = 1
for k in range(n_data):
    np.random.seed(k)
    if k%100 == 0:
        print(k)
    writelist = []
    for j in range(8):
        x = np.random.exponential(1.82,size = 80) #1/lambda = 10, 10개의 차량 , w1, w2-w1, ;.. w10 - w9
        y = []
        z = [0]*timelimit
        for i in range(100):
            temp = sum(x[:i+1])
            if temp>40:
                break;
            y.append(round(temp)) #시간차들을 더해 줘야 시간이 나옴
        carsArriveTime[j,:len(y)] = y #도착시간 행렬 8*15 행렬 (15대의 차량)
        for p in range(len(y)):
            if y[p]<timelimit:
                z[int(y[p])] = z[int(y[p])] + 1
        writelist = writelist + z

    writelist1 = writelist + getMinSignal_square(carsArriveTime, signal_num,time_of_signal)
    wr.writerow(writelist1)
f.close()

0


# Define and Train a Perceptron

In [ ]:
import numpy as np
import matplotlib.pyplot

class perceptron:
    def __init__(self, input_dim,hidden_dim,hidden_dim2,output_dim,lr=0.1):
        self.w1 = np.random.randn(input_dim,hidden_dim) / np.sqrt(input_dim/2)
        self.w2 = np.random.randn(hidden_dim, hidden_dim2) / np.sqrt(hidden_dim/2)
        self.w3 = np.random.randn(hidden_dim2, output_dim) / np.sqrt(hidden_dim2/2)
        self.h = np.zeros((1,hidden_dim))
        self.h2 = np.zeros((1,hidden_dim2))
        self.lr = lr
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.theta = 0
        self.theta2 = 0
        self.theta3 = 0
        self.regularRate = 0 #regularization
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def dsigmoid(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def softmax(x):
        y = np.exp(x)
        return y/sum(y)
    
    def relu(self,x):
        return np.maximum(0,x)
    
    def drelu(self,x):
        return np.where(x>0,1.0,0.0)
    
    def feedforward(self,x):
        a = x.astype(float)
        b = self.w1.astype(float)
        self.h = (self.sigmoid(np.dot(a,b)+self.theta))
        self.h2 = self.sigmoid(np.dot(self.h,self.w2)+self.theta2)
        return self.sigmoid(np.dot(self.h2,self.w3)+self.theta3)
    
    def bprop_w3(self,g,y):
        q = (-2)*(g-y)*self.dsigmoid(y)
        return np.dot(self.h2.reshape(self.hidden_dim2,1),q.reshape(1,self.output_dim))
    
    def bprop_w2(self,g,y):
        q1 = (-2)*(g-y)*self.dsigmoid(y)
        q2= np.dot(self.w3,q1)
        return np.dot(self.h.reshape(self.hidden_dim,1),q2*self.dsigmoid(self.h2).reshape(1,self.hidden_dim2))
    
    def bprop_w1(self,g,y,x):
        q1 = (-2)*(g-y)*self.dsigmoid(y)
        q2= np.dot(self.w3,q1)*self.dsigmoid(self.h2)
        q3 = np.dot(self.w2,q2)
        return np.dot(x.reshape(self.input_dim,1),q3*self.dsigmoid(self.h).reshape(1,self.hidden_dim))
    
    def bprop_theta3(self,g,y):
        return (2*(y-g)*self.dsigmoid(y))
    
    def bprop_theta2(self,g,y):
        q1 = (2*(y-g)*self.dsigmoid(y))
        q2 = np.dot(self.w3,q1)
        return (q2*self.dsigmoid(self.h2))
    
    def bprop_theta(self,g,y):
        q1 = 2*(y-g)*self.dsigmoid(y)
        q2= np.dot(self.w3,q1)*self.dsigmoid(self.h2)
        q3 = np.dot(self.w2,q2)
        return (q3*self.dsigmoid(self.h))
    
    def training(self,input,target):
        x = np.array(input).T
        y = self.feedforward(x)
        g = np.array(target).T
        self.w1 = self.w1*(1-self.regularRate*self.lr) - self.lr*self.bprop_w1(g,y,x) #regularization and dropout
        self.w2 = self.w2*(1-self.regularRate*self.lr) - self.lr*self.bprop_w2(g,y) 
        self.w3 = self.w3*(1-self.regularRate*self.lr) - self.lr*self.bprop_w3(g,y) 
        return np.sum((g-y)*(g-y))

input_dim=320
hidden_dim=200
hidden_dim2=100
output_dim = 32
epoch = 2

pct = perceptron(input_dim,hidden_dim,hidden_dim2,output_dim)
Losslist = []
training_dataset_file = open("input182.csv",'r') #C users mingyu폴더안

training_dataset_list = training_dataset_file.readlines()
training_dataset_file.close()


for k in range(epoch):
    print(k)
    pct.lr = pct.lr * 0.99 #learning rate decay
    Loss = 0
    for i in training_dataset_list[:]:
        all_values = i.split(',')
        inputs = (np.asfarray(all_values[:input_dim]))
        targets = np.zeros(output_dim)+0.01 #0으로 두면 피드백 자체가 안됨. 
        for j in range(4):
            targets[int(all_values[input_dim+j]) + 8*j] = 0.99
        Loss = Loss + pct.training(inputs,targets)
        all_values[input_dim+4]
    Losslist.append(Loss/len(training_dataset_list))
    


# Test with test data

In [ ]:
test_dataset_file = open("test182.csv",'r')
test_dataset_list = test_dataset_file.readlines()
test_dataset_file.close()

success = 0
failure = 0
print("testing")
for i in test_dataset_list[:]:
    all_values = i.split(',')
    targets = list(map(int,all_values[input_dim:input_dim+4]))
    inputs = (np.asfarray(all_values[:input_dim]))
    prediction_list = pct.feedforward(inputs)
    prediction = [np.argmax(prediction_list[0:8]),np.argmax(prediction_list[8:16]),np.argmax(prediction_list[16:24]),np.argmax(prediction_list[24:32])]
    if targets == prediction:
        success = success + 1
    else:
        failure = failure + 1

print("error rate = ", (failure/(success + failure)))
print(Losslist)
